<a href="https://colab.research.google.com/github/rogeriomoreirajr/Imersao_Alura_2020/blob/main/Dicion%C3%A1rio_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dando sentido aos códigos

A planilha do Inep tem os 3gb dela¹, e não é a toa: dados de 4 milhões de candidatos do Enem. É por isso que os dados são codificados, transformados em letras que, pra gente entender, precisa ir lá no Dicionário de Códigos. Isso é bom para guardar, e bom para processar os dados.

Mas tem um jeito mais fácil de "traduzir os dados"? Tem um jeito do dicionário do Inep virar um dicionário do python? Tem sim.

Abaixo eu mostro como você pode usar isso aí, no seu notebook (é só importar um arquivo). Se quiser saber como fiz o arquivo, tá na célula de baixo.

<sub>¹ E tem um jeito de lidar com ela [direto no Colab](https://colab.research.google.com/drive/15H-hIZY6WEvytKYv9v-8O9RW_vxQvFHY?usp=sharing) sem muito esforço.</sub>

In [ ]:
# for coluna in dados.columns:
import requests
import ast
import pandas as pd

# arquivo no github com o dicionário
url = 'https://raw.githubusercontent.com/rogeriomoreirajr/Imersao_Alura_2020/main/dicionario.json'

# aqui a gente busca o arquivo e recebe uma string com o dicionário
dict_string = requests.get(url).text

#com o ast.literal, a gente tranforma a string em dicionario
dict_enem = ast.literal_eval(dict_string)

# O resultado é um dicionário onde o ID da coluna é a chave para o nome 
# da variável (o que ela significa) e para as variáveis e seus significados

# Para termos um dicionário com a coluna e variáveis, um pouco de dict comprehension
dict_var = {k:v['variaveis'] for k,v in dict_enem.items() if 'variaveis' in v}

# E, numa linha, a magia. Leva um tempinho pra processar.
dados.replace(dict_var).info()

# E, claro: isso aqui é legal na hora de fazer um gráfico, 
# ou de entender os números. Mas durante o processamento, prefira
# usar os dados codificados mesmo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127380 entries, 0 to 127379
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float64(15), int64(7), object(114)
memory usage: 132.2+ MB


E, claro: isso aqui é legal na hora de fazer um gráfico, ou de entender os números. Mas durante o processamento, prefira usar os dados codificados mesmo

Mas quer saber como fiz isso? Abaixo, o código

# O código

In [ ]:
import pandas as pd
import re

path = '/content/Dicionário_Microdados_Enem_2019 - MICRODADOS_ENEM_2019.csv'

df = pd.read_csv(path, skiprows=2)

# Nenhuma novidade: abri o csv do dicionário, e pedi para pular 
# as duas primeiras linhas. Na verdade, como vão ver abaixo, só estou
# usando o pandas parar abrir o CSV e ter uma lista de listas com os valores

In [ ]:
# Crio um dicionário vazio, onde vão os valores
dict_enem = {}

# Seto a variável e o nome como nulos, para pular as primeiras linhas
variavel = ''
nome = ''

# Começo a iterar pelas linhas do dataframe
for linha in df.fillna('').values:
    # Se ele tem as duas primeiras colunas ocupadas, é o id e nome da variável

    if linha[0] and linha[1]: # Ou seja, se há valores nessas posições
        variavel = linha[0]
        nome = linha[1]
        dict_enem[variavel] = {'nome':nome}
        # insiro no dict_enem a variável e o que ela significa
    
    # Se ele tem valores nas posições 2 e 3, é o código e o significado
    # Nem todas as variáveis tem códigos e significados
    if linha[2] and linha[3]:
        # Aqui, sigo usando `variável` e `nome`. 
        # Isso lida com variáveis os vários códigos das variáveis
        
        if variavel != '': # Ou seja, não são as primeiras linhas
            if not 'variaveis' in dict_enem[variavel]:
                dict_enem[variavel]['variaveis'] = {}
            codigo = linha[2]

            # alguns códigos são int no python, mas são lidos como string
            # no csv. Aqui eu limpo isso
            try: codigo = int(codigo)
            except: pass
            
            significado = linha[3]
            dict_enem[variavel]['variaveis'][codigo] = significado

# dict_enem

{'CO_ESCOLA': {'nome': 'Código da Escola4'},
 'CO_MUNICIPIO_ESC\n': {'nome': 'Código do município da escola '},
 'CO_MUNICIPIO_NASCIMENTO': {'nome': 'Código do município de nascimento'},
 'CO_MUNICIPIO_PROVA': {'nome': 'Código do município da aplicação da prova'},
 'CO_MUNICIPIO_RESIDENCIA': {'nome': 'Código do município de residência '},
 'CO_PROVA_CH': {'nome': 'Código do tipo de prova de Ciências Humanas',
  'variaveis': {507: 'Azul',
   508: 'Amarela',
   509: 'Branca',
   510: 'Rosa',
   520: 'Laranja - Adaptada Ledor',
   524: 'Verde - Videoprova - Libras',
   547: 'Azul (Reaplicação)',
   548: 'Amarelo (Reaplicação)',
   549: 'Branco (Reaplicação)',
   550: 'Rosa (Reaplicação)',
   564: 'Laranja - Adaptada Ledor (Reaplicação)'}},
 'CO_PROVA_CN': {'nome': 'Código do tipo de prova de Ciências da Natureza',
  'variaveis': {503: 'Azul',
   504: 'Amarela',
   505: 'Cinza',
   506: 'Rosa',
   519: 'Laranja - Adaptada Ledor',
   523: 'Verde - Videoprova - Libras',
   543: 'Amarela (Rea